## Problem

A critical valve in a steam plant may be in one of two states: normal or broken.  The state of the valve changes over time according to Markovian dynamics (the next state only depends on the previous state).  The initial valve state is normal.  If the current valve state is broken, the next valve state must be broken.  If the current valve state is normal, the next valve state has a 0.2 probability of becoming broken.  We model the valve state over 3 consecutive time steps (i.e., first time step, second time step, third time step).

Note: This is not a continuous time model and we do not care about the exact times.  We simply have 3 different discrete random variables for valve state corresponding to the valve state at 3 consecutive time steps.

A very noisy sensor monitors the valve state at every time step.  The sensor may take on one of two assignments: observed_normal or observed_broken.  If the valve state is normal, there is a uniform (i.e., equal) probability of the sensor reading either observed_normal or observed_broken.  If the valve state is broken, then the sensor will always read observed_broken.

Answer the following questions, by showing the code that executes the query and produces the exact numerical result:

1. Given no evidence, what is the probability that the valve state is broken at the third time step?
2. Given the evidence observed_broken for the sensor at the second time step, what is the probability that the valve state is broken at the third time step?
3. Given the sensor observation sequence (observed_broken, observed_normal, observed_broken) as evidence, what is the probability that the valve state is broken at the third time step?
4. Given the sensor observation sequence (observed_broken, observed_broken, observed_broken) as evidence, what is the probability that the valve state is broken at the third time step?
5. Given the evidence that the valve state at the second time step is normal, what is the probability of observing sensor value observed_normal at the third time step?


In [3]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD as cpd
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
import pandas as pd
import numpy as np

### Build the Baysian Network

In [22]:
valve = BayesianModel([('v1', 'v2'), 
                       ('v2', 'v3'),
                       ('v1', 's1'),
                       ('v2', 's2'),
                       ('v3', 's3')])

v_v_values = [[1, 0.2], [0, 0.8]]
v_s_values = [[1, 0.5], [0, 0.5]]

cpd_v1 = cpd(variable='v1', variable_card=2,values=[[0], [1]])
cpd_v2 = cpd(variable='v2', variable_card=2,
            values=v_v_values,
            evidence=['v1'],
            evidence_card=[2])
cpd_v3 = cpd(variable='v3', variable_card=2,
            values=v_v_values,
            evidence=['v2'],
            evidence_card=[2])
cpd_s1 = cpd(variable='s1', variable_card=2,
            values=v_s_values,
            evidence=['v1'],
            evidence_card=[2])
cpd_s2 = cpd(variable='s2', variable_card=2,
            values=v_s_values,
            evidence=['v2'],
            evidence_card=[2])
cpd_s3 = cpd(variable='s3', variable_card=2,
            values=v_s_values,
            evidence=['v3'],
            evidence_card=[2])


In [23]:
valve.add_cpds(cpd_v1, cpd_v2, cpd_v3, cpd_s1, cpd_s2, cpd_s3)
valve.check_model()

True

In [24]:
valve.get_independencies()

(s2 _|_ s3, s1, v1, v3 | v2)
(s2 _|_ s1 | v1)
(s2 _|_ s3 | v3)
(s2 _|_ s1, v1, v3 | s3, v2)
(s2 _|_ s3, v1, v3 | s1, v2)
(s2 _|_ s3, s1, v3 | v1, v2)
(s2 _|_ s3, s1, v1 | v3, v2)
(s2 _|_ s1 | s3, v1)
(s2 _|_ s3 | s1, v3)
(s2 _|_ s3, s1 | v1, v3)
(s2 _|_ v1, v3 | s3, s1, v2)
(s2 _|_ s1, v3 | s3, v1, v2)
(s2 _|_ s1, v1 | v3, s3, v2)
(s2 _|_ s3, v3 | s1, v1, v2)
(s2 _|_ s3, v1 | v3, s1, v2)
(s2 _|_ s3, s1 | v3, v1, v2)
(s2 _|_ s1 | s3, v1, v3)
(s2 _|_ s3 | s1, v1, v3)
(s2 _|_ v3 | s3, s1, v1, v2)
(s2 _|_ v1 | v3, s3, s1, v2)
(s2 _|_ s1 | v3, s3, v1, v2)
(s2 _|_ s3 | v3, s1, v1, v2)
(s1 _|_ s2, s3, v3 | v2)
(s1 _|_ v2, s2, s3, v3 | v1)
(s1 _|_ s3 | v3)
(s1 _|_ s3, v3 | s2, v2)
(s1 _|_ s2, v3 | s3, v2)
(s1 _|_ s2, s3, v3 | v1, v2)
(s1 _|_ s2, s3 | v3, v2)
(s1 _|_ v2, s3, v3 | s2, v1)
(s1 _|_ s3 | s2, v3)
(s1 _|_ v2, s2, v3 | s3, v1)
(s1 _|_ s2, s3, v2 | v1, v3)
(s1 _|_ v3 | s2, s3, v2)
(s1 _|_ s3, v3 | s2, v1, v2)
(s1 _|_ s3 | s2, v3, v2)
(s1 _|_ s2, v3 | s3, v1, v2)
(s1 _|_ s2 | v3, s3, v2

### Run queries

1. Given no evidence, what is the probability that the valve state is broken at the third time step?

Answer: 0.36

In [25]:
infer = VariableElimination(valve)
q1 = infer.query(variables=['v3'])
print(q1['v3'])

╒══════╤═══════════╕
│ v3   │   phi(v3) │
╞══════╪═══════════╡
│ v3_0 │    0.3600 │
├──────┼───────────┤
│ v3_1 │    0.6400 │
╘══════╧═══════════╛


2. Given the evidence observed_broken for the sensor at the second time step, what is the probability that the valve state is broken at the third time step?

Answer: 0.4667

In [26]:
q2 = infer.query(variables=['v3'], evidence={'s2':0})
print(q2['v3'])

╒══════╤═══════════╕
│ v3   │   phi(v3) │
╞══════╪═══════════╡
│ v3_0 │    0.4667 │
├──────┼───────────┤
│ v3_1 │    0.5333 │
╘══════╧═══════════╛


3. Given the sensor observation sequence (observed_broken, observed_normal, observed_broken) as evidence, what is the probability that the valve state is broken at the third time step?

Answer: 0.333

In [27]:
q3 = infer.query(variables=['v3'], evidence={'s1':0, 's2':1, 's3':0})
print(q3['v3'])

╒══════╤═══════════╕
│ v3   │   phi(v3) │
╞══════╪═══════════╡
│ v3_0 │    0.3333 │
├──────┼───────────┤
│ v3_1 │    0.6667 │
╘══════╧═══════════╛


4. Given the sensor observation sequence (observed_broken, observed_broken, observed_broken) as evidence, what is the probability that the valve state is broken at the third time step?

Answer: 0.6364

In [28]:
q4 = infer.query(variables=['v3'], evidence={'s1':0, 's2':0, 's3':0})
print(q4['v3'])

╒══════╤═══════════╕
│ v3   │   phi(v3) │
╞══════╪═══════════╡
│ v3_0 │    0.6364 │
├──────┼───────────┤
│ v3_1 │    0.3636 │
╘══════╧═══════════╛


5. Given the evidence that the valve state at the second time step is normal, what is the probability of observing sensor value observed_normal at the third time step?

Answer: 0.4

In [29]:
q5 = infer.query(variables=['s3'], evidence={'v2':1})
print(q5['s3'])

╒══════╤═══════════╕
│ s3   │   phi(s3) │
╞══════╪═══════════╡
│ s3_0 │    0.6000 │
├──────┼───────────┤
│ s3_1 │    0.4000 │
╘══════╧═══════════╛
